# Notebook 5: Agents & MCP Tools

This notebook is for developers who are already familiar with [basic agent workflows](4.1_tool_calling.ipynb). 
Here, we will highlight more advanced use cases for agents where a single tool call is insufficient to complete the required task.

We will also use [MCP tools](https://github.com/modelcontextprotocol/servers) (which can be deployed onto an OpenShift cluster) throughout this demo to show users how to extend their agents beyond Llama Stacks's current builtin tools and connect to many different services and data sources to build their own custom agents.  

### Agent Example:

This notebook will walkthrough how to build a system that can answer the following question via an agent built with Llama Stack:

- *"Review OpenShift logs for the failing-pod. Categorize each as either ‘Normal’ or ‘Error’. If it's an error search for a solution. Summarize any errors found."*

### MCP Tools:

#### OpenShift MCP Server

Throughout this notebook we will be relying on the [Kubernetes MCP server](https://github.com/manusa/kubernetes-mcp-server) by [manusa](https://github.com/manusa) to interact with our OpenShift cluster.

#### Github MCP Server

Throughout this notebook we will be relying on the [Github MCP server](https://github.com/github/github-mcp-server) by Github to interact with our Github repository.

## Setting Up this Notebook
We will initialize our environment as described in detail in our ["Llama Stack" notebook](./3_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [ ]:
from src.client_tools import get_pod_log
from src import lab
from src.lab import get_llama_stack_client, get_react_agent, run_session

## Validate tools are available in our Llama Stack instance

When an instance of Llama Stack is redeployed, it may be the case that the tools will need to be re-registered. Also if a tool is already registered with a Llama Stack instance, trying to register another one with the same `toolgroup_id` will throw you an error.

For this reason, it is recommended to validate your tools and toolgroups. The following code will check that both the `builtin::websearch` and `mcp::openshift` tools are correctly registered, and if not it will attempt to register them using their specific endpoints.

In [ ]:
client = get_llama_stack_client()
registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]
if "mcp::openshift" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::openshift",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":lab.ocp_mcp_url},
    )

print(f"Your Llama Stack server is registered with the following tool groups @ {set(registered_toolgroups)} \n")

## Defining our Agent - ReAct

Now that we've shown that we can successfully accomplish this multi-step multi-tool task using prompt chaining, let's see if we can give our agent a bit more autonomy to perform the same task but with a single prompt instead of a chain. To do this, we will instantiate a **ReAct agent** (which is included in the llama stack python client by default).The ReAct agent is a variant of the simple agent but with the ability to loop through "Reason then Act" iterations, thinking through the problem and then using tools until it determines that it's task has been completed successfully.  

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

Below you will see the slight differences in the agent definition and the prompt used to accomplish our task.

In [ ]:
tools=["mcp::openshift", "builtin::websearch", 'mcp::github']
agent = get_react_agent(tools)
user_prompts =[
    """
    f"There's a pod {pod_name} in namespace {namespace} with a container {container_name}"
    f" that's been failing. Find out why it's been failing and provide a troubleshooting"
    f" recommendation based on public knowledge."
    """
]
run_session(agent, 'mcp-session', user_prompts)

### Output Analysis

Above, we can see that the ReAct agent took nearly an identical approach to the prompt chaining method above, but using a single prompt instead of a chain.  

1. First the LLM generated a tool call for the `pods_log` tool included in the **OpenShift MCP server** and fetched the logs for the specified pod.
2. The tool successfully retrieved the logs for the pod.
3. The LLM  then received the logs from the tool call, along with the original query.
4. This context was then passed back to the LLM for the final inference. The inference result provided a summary of the pod logs along with its category of 'Normal' or 'Error'.
5. Next the LLM generates a tool call for the default builtin `brave_search` tool which was not available. This is because the models have been trained with Brave Search as a built-in tool.
6. Next the LLM generates a tool call for the `web_search` tool instead looking for the top answer to the error.
7. A summary of the pod log error and possible next steps to help solve the problem are suggested. 

## Key Takeaways

This notebook demonstrated how to build an agentic MCP applications with Llama Stack. We did this by initializing an agent with access to two MCP servers that were registered to our Llama Stack server, then invoked the agent on our specified set of queries. We showed that we can do this with more directed Prompt Chaining or with the more open ended ReAct pattern.